In [ ]:
import json
from typing import Dict, Any, List, Tuple

###############################################################################
# Helpers to parse phases (like "S1901M") into a sorted, friendly label
###############################################################################

"""
Results JSON structure:

json_skeleton = {
    "game_id": "",
    "env_state": {
        "timestamp": 0,
        "zobrist_hash": "",
        "note": "",
        "name": "",
        "units": {
            "AUSTRIA": [],
            "ENGLAND": [],
            "FRANCE": [],
            "GERMANY": [],
            "ITALY": [],
            "RUSSIA": [],
            "TURKEY": []
        },
        "retreats": {
            "AUSTRIA": {},
            "ENGLAND": {},
            "FRANCE": {},
            "GERMANY": {},
            "ITALY": {},
            "RUSSIA": {},
            "TURKEY": {}
        },
        "centers": {
            "AUSTRIA": [],
            "ENGLAND": [],
            "FRANCE": [],
            "GERMANY": [],
            "ITALY": [],
            "RUSSIA": [],
            "TURKEY": []
        },
        "homes": {
            "AUSTRIA": [],
            "ENGLAND": [],
            "FRANCE": [],
            "GERMANY": [],
            "ITALY": [],
            "RUSSIA": [],
            "TURKEY": []
        },
        "influence": {
            "AUSTRIA": [],
            "ENGLAND": [],
            "FRANCE": [],
            "GERMANY": [],
            "ITALY": [],
            "RUSSIA": [],
            "TURKEY": []
        },
        "civil_disorder": {
            "AUSTRIA": 0,
            "ENGLAND": 0,
            "FRANCE": 0,
            "GERMANY": 0,
            "ITALY": 0,
            "RUSSIA": 0,
            "TURKEY": 0
        },
        "builds": {
            "AUSTRIA": {
                "count": 0,
                "homes": []
            },
            "ENGLAND": {
                "count": 0,
                "homes": []
            },
            "FRANCE": {
                "count": 0,
                "homes": []
            },
            "GERMANY": {
                "count": 0,
                "homes": []
            },
            "ITALY": {
                "count": 0,
                "homes": []
            },
            "RUSSIA": {
                "count": 0,
                "homes": []
            },
            "TURKEY": {
                "count": 0,
                "homes": []
            }
        }
    },
    "done": False,
    "agents_data": {
        "AUT": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-ENG": "",
                "AUT-FRA": "",
                "AUT-GER": "",
                "AUT-ITA": "",
                "AUT-RUS": "",
                "AUT-TUR": ""
            }
        },
        "ENG": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-ENG": "",
                "ENG-FRA": "",
                "ENG-GER": "",
                "ENG-ITA": "",
                "ENG-RUS": "",
                "ENG-TUR": ""
            }
        },
        "FRA": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-FRA": "",
                "ENG-FRA": "",
                "FRA-GER": "",
                "FRA-ITA": "",
                "FRA-RUS": "",
                "FRA-TUR": ""
            }
        },
        "GER": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-GER": "",
                "ENG-GER": "",
                "FRA-GER": "",
                "GER-ITA": "",
                "GER-RUS": "",
                "GER-TUR": ""
            }
        },
        "ITA": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-ITA": "",
                "ENG-ITA": "",
                "FRA-ITA": "",
                "GER-ITA": "",
                "ITA-RUS": "",
                "ITA-TUR": ""
            }
        },
        "RUS": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-RUS": "",
                "ENG-RUS": "",
                "FRA-RUS": "",
                "GER-RUS": "",
                "ITA-RUS": "",
                "RUS-TUR": ""
            }
        },
        "TUR": {
            "personality": "",
            "goals": [],
            "journal": [],
            "model_name": "",
            "relationships": {
                "AUT-TUR": "",
                "ENG-TUR": "",
                "FRA-TUR": "",
                "GER-TUR": "",
                "ITA-TUR": "",
                "RUS-TUR": ""
            }
        }
    },
    "turn_history": [
        {
            "phase": "",
            "env_state": {},
            "issued_orders": {},
            "accepted_orders": {},
            "player_relationships": {}
        }
    ],
    "negotiation_history": [
        {
            "turn_index": 0,
            "subrounds": [
                {
                    "subround_index": 0,
                    "sent_missives": [
                        {"sender": "", "recipients": [], "body": ""}
                    ],
                    "received_missives": {
                        "AUT": [],
                        "ENG": [],
                        "FRA": [],
                        "GER": [],
                        "ITA": [],
                        "RUS": [],
                        "TUR": []
                    }
                }
            ],
            "final_summaries": {
                "AUT": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                },
                "ENG": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                },
                "FRA": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                },
                "GER": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                },
                "ITA": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                },
                "RUS": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                },
                "TUR": {
                    "journal_summary": "",
                    "intent": "",
                    "rship_updates": []
                }
            }
        }
    ]
}

"""


def parse_phase(phase_str: str) -> Tuple[int,int,str]:
    """
    Attempt to parse a phase like "S1901M" or "F1901A" or "W1902R"
    Return (year, season_order, pretty_label)
      season_order: 1=Spring, 2=Fall, 3=Winter (arbitrary choice)
      pretty_label: "1901 Spring", "1901 Fall", "1901 Winter"
    If parsing fails, fallback to (9999, 9, phase_str).
    """
    if len(phase_str) < 5:
        return (9999, 9, phase_str)

    season_char = phase_str[0]
    year_str = phase_str[1:5]
    try:
        year = int(year_str)
    except ValueError:
        return (9999, 9, phase_str)

    if season_char == 'S':
        season_txt = "Spring"
        season_order = 1
    elif season_char == 'F':
        season_txt = "Fall"
        season_order = 2
    elif season_char == 'W':
        season_txt = "Winter"
        season_order = 3
    else:
        season_txt = "Unknown"
        season_order = 9

    pretty_label = f"{year} {season_txt}"
    return (year, season_order, pretty_label)

def relationship_to_numeric(relationship_str: str) -> int:
    """
    Convert the relationship string into numeric form:
      "++" =>  2
      "+"  =>  1
      "~"  =>  0
      "-"  => -1
      "--" => -2
    """
    mapping = {"++": 2, "+": 1, "~": 0, "-": -1, "--": -2}
    return mapping.get(relationship_str, 0)

###############################################################################
# 1) Parse stats for AUT, sorted by chronological phase
###############################################################################
def parse_aut_statistics(game_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Return an OrderedDict-like structure (phase_label -> stats).
    We'll sort phases by chronological order. Stats:
      {
        "centers_count": int,
        "units_count": int,
        "influence_count": int,
        "builds_count": int,
        "civil_disorder": int,
        "relationships_self": { "AUT-ENG": int, ... },
        "relationships_others": { "ENG-AUT": int, ... },
        "phase_str": the raw string from JSON,
        "phase_label": "1901 Spring" etc
      }
    """
    turn_history = game_data.get("turn_history", [])
    info_list = []
    for turn in turn_history:
        phase_str = turn.get("phase", "UnknownPhase")
        (year, season_ord, nice_label) = parse_phase(phase_str)

        env_state = turn.get("env_state", {})
        centers_aut = env_state.get("centers", {}).get("AUSTRIA", [])
        units_aut = env_state.get("units", {}).get("AUSTRIA", [])
        influence_aut = env_state.get("influence", {}).get("AUSTRIA", [])
        builds_data = env_state.get("builds", {}).get("AUSTRIA", {})
        build_count = builds_data.get("count", 0)
        civ_dis = env_state.get("civil_disorder", {}).get("AUSTRIA", 0)

        # relationships
        relationships_self = {}
        relationships_others = {}

        pr = turn.get("player_relationships", {})
        aut_rel = pr.get("AUT", {})
        for rk, rv in aut_rel.items():
            relationships_self[rk] = relationship_to_numeric(rv)

        for power, rel_map in pr.items():
            if power == "AUT":
                continue
            for rk, rv in rel_map.items():
                if "AUT" in rk:
                    relationships_others[rk] = relationship_to_numeric(rv)

        info_list.append({
            "phase": phase_str,
            "phase_label": nice_label,
            "sort_key": (year, season_ord),
            "centers_count": len(centers_aut),
            "units_count": len(units_aut),
            "influence_count": len(influence_aut),
            "builds_count": build_count,
            "civil_disorder": civ_dis,
            "relationships_self": relationships_self,
            "relationships_others": relationships_others
        })

    info_list.sort(key=lambda x: x["sort_key"])

    final_dict = {}
    label_count = {}
    for entry in info_list:
        plabel = entry["phase_label"]
        if plabel in final_dict:
            label_count[plabel] = label_count.get(plabel, 1) + 1
            suffix = f"({label_count[plabel]})"
            plabel = f"{plabel}{suffix}"
        else:
            label_count[plabel] = 1
        final_dict[plabel] = entry

    return final_dict

###############################################################################
# 2) Parse negotiations for AUT, also sorted by turn_index, 
#    and label them with the matching phase label if possible
###############################################################################
def parse_aut_negotiations(game_data: Dict[str, Any], stats_dict: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    negotiation_history: each item is { "turn_index": int, ... } 
    We'll find the corresponding turn_history[turn_index] for the phase => label
    Then produce a structure like:
      [
        {
          "turn_index": X,
          "phase_label": "1901 Spring",
          "subrounds": [...],
          "final_summary": { ... }
        }, ...
      ]
    Sorted by turn_index ascending.
    """
    neg_data = []
    negotiation_history = game_data.get("negotiation_history", [])
    negotiation_history.sort(key=lambda x: x.get("turn_index", -1))
    
    year_counts = {}
    
    for turn_neg in negotiation_history:
        tindex = turn_neg.get("turn_index", -1)
        
        if "phase" in turn_neg:
            phase_str = turn_neg["phase"]
            _, _, nice_label = parse_phase(phase_str)
        else:
            year = 1901 + (tindex // 2)
            if year not in year_counts:
                year_counts[year] = 0
                season = "Spring"
            else:
                year_counts[year] += 1
                if year_counts[year] % 2 == 0:
                    season = "Spring"
                else:
                    season = "Fall"
            phase_str = f"{'S' if season == 'Spring' else 'F'}{year}M"
            nice_label = f"{year} {season}"
        
        subrounds = turn_neg.get("subrounds", [])
        final_summaries = turn_neg.get("final_summaries", {})
        aut_summary = final_summaries.get("AUT", {})

        sr_list = []
        for sr in subrounds:
            sr_idx = sr.get("subround_index", -1)
            sent_all = sr.get("sent_missives", [])
            sent_aut = [m for m in sent_all if m.get("sender") == "AUT"]
            recv_aut = sr.get("received_missives", {}).get("AUT", [])
            sr_list.append({
                "subround_index": sr_idx,
                "sent_by_aut": sent_aut,
                "received_by_aut": recv_aut
            })

        neg_data.append({
            "turn_index": tindex,
            "phase": phase_str,
            "phase_label": nice_label,
            "subrounds": sr_list,
            "final_summary": {
                "journal_summary": aut_summary.get("journal_summary", ""),
                "intent": aut_summary.get("intent", ""),
                "rship_updates": aut_summary.get("rship_updates", [])
            }
        })

    return neg_data

###############################################################################
# 3) HTML generation with Bootstrap + Chart.js
###############################################################################
def generate_html_report(
    game_data: Dict[str, Any],
    stats_for_aut: Dict[str, Any], 
    negotiations_data: List[Dict[str, Any]]
) -> str:
    """
    - stats_for_aut is an ordered dict-like: { "1901 Spring": { centers_count, ...}, "1901 Fall": {...}, ... }
    - negotiations_data is a list sorted by turn_index, each with a known phase_label.
    """
    import math
    import os
    import glob
    
    # --------------------------------------------------------
    # HELPER to turn "F1907M" into "1907 Fall Movement", etc.
    # --------------------------------------------------------
    def make_friendly_label(raw_phase: str) -> str:
        # raw_phase could be "S1901M", "F1907A", "W1905R", etc.
        # parse_phase() gives us (year, season_order, "1907 Fall")
        # then we can read the trailing character for M=Movement, A=Adjustment, R=Retreat, etc.
        (yr, _, season_str) = parse_phase(raw_phase)
        suffix = raw_phase[5:].strip()  # e.g. "M", "A", "R"...

        # If there's more than one trailing char (e.g. "MA"? Rare), handle gracefully
        # Typically you see a single letter but let's handle multiple:
        # For example, "M" => Movement, "A" => Adjustment, "R" => Retreat
        # If multiple, just join them; or you can expand as needed.
        suffix_map = {
            'M': 'Movement',
            'A': 'Adjustment',
            'R': 'Retreat',
            # etc. If you have others like 'B' or 'D' you can add them here
        }
        # Convert each character in suffix into a descriptive label if known, otherwise keep it
        suffix_str_parts = []
        for ch in suffix:
            suffix_str_parts.append(suffix_map.get(ch, ch))
        suffix_str = " ".join(suffix_str_parts).strip()

        # result example: "1907 Fall Movement"
        # if suffix_str is empty, we’ll just show e.g. "1907 Fall"
        final_label = season_str
        if suffix_str:
            final_label = f"{season_str} {suffix_str}"

        return f"{yr} {final_label}".strip()

    # --------------------------------------------------------
    # (1) Possibly gather raw phases in a list that lines up with stats_for_aut’s order
    #     So our x-axis is something like ["F1907M","W1907R",...] but displayed readably.
    # --------------------------------------------------------
    
    # Pull the sorted "display keys" from stats_for_aut
    display_keys = list(stats_for_aut.keys())  # e.g. ["1901 Spring", "1901 Fall", ...] but in sorted order
    # We will extract the actual raw_phase from each stats entry:
    raw_phases = []
    friendly_phases = []

    for disp_key in display_keys:
        raw_p = stats_for_aut[disp_key]["phase"]  # e.g. "F1907M"
        raw_phases.append(raw_p)
        friendly_phases.append(make_friendly_label(raw_p))
    
    # --------------------------------------------------------
    # (2) Gather all the metric arrays in that same order
    # --------------------------------------------------------
    centers_arr = []
    units_arr = []
    influence_arr = []
    builds_arr = []
    civdis_arr = []

    for disp_key in display_keys:
        d = stats_for_aut[disp_key]
        centers_arr.append(d["centers_count"])
        units_arr.append(d["units_count"])
        influence_arr.append(d["influence_count"])
        builds_arr.append(d["builds_count"])
        civdis_arr.append(d["civil_disorder"])
    
    # For relationships
    all_self_keys = set()
    all_others_keys = set()
    for disp_key in display_keys:
        d = stats_for_aut[disp_key]
        all_self_keys.update(d["relationships_self"].keys())
        all_others_keys.update(d["relationships_others"].keys())

    all_self_keys = sorted(list(all_self_keys))
    all_others_keys = sorted(list(all_others_keys))

    self_rels_data = {k: [] for k in all_self_keys}
    others_rels_data = {k: [] for k in all_others_keys}

    self_agg_arr = []
    others_agg_arr = []

    for disp_key in display_keys:
        d = stats_for_aut[disp_key]
        # self
        rs_self = d["relationships_self"]
        if rs_self:
            sum_val = 0
            for k in all_self_keys:
                v = rs_self.get(k, 0)
                self_rels_data[k].append(v)
                sum_val += v
            avg_self = sum_val / len(all_self_keys)
        else:
            for k in all_self_keys:
                self_rels_data[k].append(0)
            avg_self = 0
        self_agg_arr.append(avg_self)

        # others
        rs_others = d["relationships_others"]
        if rs_others:
            sum_val = 0
            for k in all_others_keys:
                v = rs_others.get(k, 0)
                others_rels_data[k].append(v)
                sum_val += v
            avg_oth = sum_val / len(all_others_keys)
        else:
            for k in all_others_keys:
                others_rels_data[k].append(0)
            avg_oth = 0
        others_agg_arr.append(avg_oth)

    # top-level sums
    def avg_of(arr):
        return sum(arr) / len(arr) if arr else 0
    def last_of(arr):
        return arr[-1] if arr else 0

    avg_centers = avg_of(centers_arr)
    final_centers = last_of(centers_arr)
    avg_units = avg_of(units_arr)
    final_units = last_of(units_arr)
    avg_influence = avg_of(influence_arr)
    final_influence = last_of(influence_arr)
    avg_selfr = avg_of(self_agg_arr)
    final_selfr = last_of(self_agg_arr)
    avg_othr = avg_of(others_agg_arr)
    final_othr = last_of(others_agg_arr)

    # --------------------------------------------------------
    # (3) Collect Orders (Issued / Valid / Rejected) for each raw phase
    # --------------------------------------------------------
    orders_issued_arr = []
    orders_valid_arr = []
    orders_rejected_arr = []
    turn_history = game_data.get("turn_history", [])

    for rp in raw_phases:
        # find the corresponding turn
        matching_turn = None
        for th in turn_history:
            if th.get("phase") == rp:
                matching_turn = th
                break
        if matching_turn:
            all_issued = matching_turn.get("issued_orders", {}).get("AUT", [])
            all_accepted = matching_turn.get("accepted_orders", {}).get("AUT", [])
            n_issued = len(all_issued)
            n_valid = len(all_accepted)
            n_rejected = n_issued - n_valid
        else:
            n_issued = 0
            n_valid = 0
            n_rejected = 0

        orders_issued_arr.append(n_issued)
        orders_valid_arr.append(n_valid)
        orders_rejected_arr.append(n_rejected)

    # --------------------------------------------------------
    # (4) Start building HTML
    # --------------------------------------------------------
    import html as html_lib  # if needed for escaping
    import urllib
    import sys

    html = []
    html.append("<!DOCTYPE html><html lang='en'><head>")
    html.append("<meta charset='utf-8'/>")
    html.append("<meta name='viewport' content='width=device-width, initial-scale=1'/>")
    html.append("<title>Diplomacy - AUT Report</title>")

    # Bootstrap + Chart.js
    html.append("<link href='https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css' rel='stylesheet'/>")
    html.append("<script src='https://cdn.jsdelivr.net/npm/chart.js'></script>")
    html.append("<style>")
    html.append("""
    .svg-player-container {
        position: relative;
        width: 100%;
        max-width: 1000px;
        margin: 0 auto;
        background-color: #f8f9fa;
        border-radius: 8px;
        overflow: hidden;
    }
    .svg-display {
        width: 100%;
        height: 600px;
        background-color: white;
        display: flex;
        align-items: center;
        justify-content: center;
        overflow: hidden;
    }
    .svg-display object, .svg-display img {
        max-width: 100%;
        max-height: 100%;
        object-fit: contain;
    }
    .player-controls {
        padding: 15px;
        background-color: #e9ecef;
        display: flex;
        align-items: center;
        gap: 10px;
    }
    .progress-container {
        flex-grow: 1;
        height: 10px;
        background-color: #ced4da;
        border-radius: 5px;
        position: relative;
        cursor: pointer;
    }
    .progress-bar {
        height: 100%;
        background-color: #0d6efd;
        border-radius: 5px;
        width: 0%;
    }
    .phase-label {
        font-weight: bold;
        min-width: 120px;
        text-align: center;
    }
    .control-button {
        background-color: #6c757d;
        color: white;
        border: none;
        border-radius: 4px;
        padding: 5px 10px;
        cursor: pointer;
    }
    .control-button:hover {
        background-color: #5a6268;
    }
    .control-button:disabled {
        background-color: #adb5bd;
        cursor: not-allowed;
    }
    """)
    html.append("</style>")
    html.append("</head><body class='bg-light'>")
    html.append("<div class='container mt-4'>")
    html.append("<h1>Diplomacy Report for Austria (AUT)</h1>")

    # Basic info
    game_id = game_data.get("game_id","Unknown")
    done_flag = game_data.get("done", False)
    html.append(f"<h4><strong>Game ID:</strong> {game_id}</h4>")    

    # Overall Stats
    html.append("""
    <div class="card mb-3">
      <div class="card-body">
        <h4>Overall Stats</h4>
    """)
    html.append("<ul>")
    html.append(f"<li><b>Avg Centers:</b> {avg_centers:.2f}, <b>Final:</b> {final_centers}</li>")
    html.append(f"<li><b>Avg Units:</b> {avg_units:.2f}, <b>Final:</b> {final_units}</li>")
    html.append(f"<li><b>Avg Influence:</b> {avg_influence:.2f}, <b>Final:</b> {final_influence}</li>")
    html.append(f"<li><b>Avg Self-Relationship:</b> {avg_selfr:.2f}, <b>Final:</b> {final_selfr:.2f}</li>")
    html.append(f"<li><b>Avg Others-Relationship:</b> {avg_othr:.2f}, <b>Final:</b> {final_othr:.2f}</li>")
    html.append("</ul>")
    html.append("""
      </div>
    </div>
    """)

    # Tabs
    svg_files = []
    run_id = game_id
    svg_dir = f"gamestate_renders/{run_id}"
    if os.path.exists(svg_dir):
        import glob
        svg_pattern = os.path.join(svg_dir, "gamestate_*.svg")
        svg_files = glob.glob(svg_pattern)

    html.append("""
<ul class="nav nav-tabs" id="myTab" role="tablist">
  <li class="nav-item" role="presentation">
    <button class="nav-link active" id="stats-tab" data-bs-toggle="tab" data-bs-target="#statsTabContent" type="button" role="tab" aria-controls="statsTabContent" aria-selected="true">AUT Charts</button>
  </li>
  <li class="nav-item" role="presentation">
    <button class="nav-link" id="negotiation-tab" data-bs-toggle="tab" data-bs-target="#negotiationTabContent" type="button" role="tab" aria-controls="negotiationTabContent" aria-selected="false">Negotiations</button>
  </li>
""")
    if svg_files:
        html.append("""  
  <li class="nav-item" role="presentation">
    <button class="nav-link" id="animation-tab" data-bs-toggle="tab" data-bs-target="#animationTabContent" type="button" role="tab" aria-controls="animationTabContent" aria-selected="false">Game Animation</button>
  </li>
""")
    html.append("""
</ul>
<div class="tab-content" id="myTabContent">
  <!-- STATS TAB -->
  <div class="tab-pane fade show active p-3" id="statsTabContent" role="tabpanel" aria-labelledby="stats-tab">
""")

    # Main Metrics
    html.append("<h3>Main Metrics</h3>")
    html.append("<canvas id='mainMetricsChart' width='800' height='300'></canvas>")

    # Self Rels
    html.append("<hr/><h3>Relationships (Self-Perceived)</h3>")
    html.append("<canvas id='selfRelsChart' width='800' height='300'></canvas>")

    # Others Rels
    html.append("<hr/><h3>Relationships (Others → AUT)</h3>")
    html.append("<canvas id='othersRelsChart' width='800' height='300'></canvas>")

    # Orders Chart
    html.append("<hr/><h3>Orders: Issued vs Valid vs Rejected</h3>")
    html.append("<canvas id='ordersChart' width='800' height='300'></canvas>")

    html.append("</div>")  # end statsTabContent

    # Negotiation Tab
    html.append("""
  <div class="tab-pane fade p-3" id="negotiationTabContent" role="tabpanel" aria-labelledby="negotiation-tab">
    <h2>Negotiations / Dialogues</h2>
""")
    # Show negotiations
    html.append('<div class="accordion" id="negAccordion">')
    for i, nd in enumerate(negotiations_data):
        tindex = nd["turn_index"]
        ph_label = nd["phase_label"] if nd["phase_label"] else nd["phase"]
        subrounds = nd["subrounds"]
        final_sum = nd["final_summary"]

        heading_id = f"negHeading{i}"
        collapse_id = f"negCollapse{i}"

        title_str = f"Negotiation Turn {tindex} ({ph_label})"
        html.append(f"""
  <div class="accordion-item">
    <h2 class="accordion-header" id="{heading_id}">
      <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#{collapse_id}" aria-expanded="false" aria-controls="{collapse_id}">
        {title_str}
      </button>
    </h2>
    <div id="{collapse_id}" class="accordion-collapse collapse" aria-labelledby="{heading_id}" data-bs-parent="#negAccordion">
      <div class="accordion-body">
""")
        # Subrounds
        for sr_obj in subrounds:
            sr_idx = sr_obj["subround_index"]
            sents = sr_obj["sent_by_aut"]
            recvs = sr_obj["received_by_aut"]

            html.append(f"<div class='mb-2'><b>Subround {sr_idx}</b></div>")
            if sents:
                html.append("<div class='ms-4'><u>Sent by AUT:</u><ul>")
                for msg in sents:
                    to_ = ", ".join(msg.get("recipients", []))
                    body_ = msg.get("body","")
                    html.append(f"<li><b>To</b> [{to_}]: {body_}</li>")
                html.append("</ul></div>")

            if recvs:
                html.append("<div class='ms-4'><u>Received by AUT:</u><ul>")
                for msg in recvs:
                    sender_ = msg.get("sender","")
                    body_ = msg.get("body","")
                    html.append(f"<li><b>From</b> [{sender_}]: {body_}</li>")
                html.append("</ul></div>")

        html.append("<hr/>")
        html.append("<p><b>Final Summary (AUT):</b></p>")
        html.append("<ul>")
        jsum = final_sum["journal_summary"]
        jintent = final_sum["intent"]
        jrship = final_sum["rship_updates"]
        html.append(f"<li><i>Journal Summary:</i> {jsum}</li>")
        html.append(f"<li><i>Intent:</i> {jintent}</li>")
        if jrship:
            html.append(f"<li><i>Relationship Updates:</i> {jrship}</li>")
        html.append("</ul>")

        html.append("""
      </div>
    </div>
  </div>
""")
    html.append("</div>")  # end accordion
    html.append("</div>")  # end negotiationTabContent

    # Animation tab
    if svg_files:
        # Sort them to match chronological order
        def extract_phase_from_filename(filename):
            import os
            base = os.path.basename(filename)
            ph = base.replace("gamestate_","").replace(".svg","")
            (y, s, _) = parse_phase(ph)
            return (y, s)
        svg_files.sort(key=extract_phase_from_filename)

        # Build JS arrays
        import os
        svg_paths_js = "[" + ",".join(f"'{path}'" for path in svg_files) + "]"
        # Also build friendly labels for each:
        svg_phases = []
        for f in svg_files:
            base = os.path.basename(f)
            rp = base.replace("gamestate_","").replace(".svg","")
            svg_phases.append(make_friendly_label(rp))
        svg_phases_js = "[" + ",".join(f"'{x}'" for x in svg_phases) + "]"

        html.append("""
  <div class="tab-pane fade p-3" id="animationTabContent" role="tabpanel" aria-labelledby="animation-tab">
    <h2>Game Animation</h2>
    <div class="svg-player-container">
      <div class="svg-display" id="svgDisplay">
      </div>
      <div class="player-controls">
        <button class="control-button" id="prevBtn">◀</button>
        <button class="control-button" id="playPauseBtn">▶</button>
        <button class="control-button" id="nextBtn">▶</button>
        <div class="progress-container" id="progressContainer">
          <div class="progress-bar" id="progressBar"></div>
        </div>
        <div class="phase-label" id="phaseLabel"></div>
      </div>
    </div>
  </div>
""")

    html.append("</div>")  # end .tab-content
    html.append("</div>")  # end .container

    # --------------------------------------------------------
    # Chart.js generation
    # --------------------------------------------------------
    html.append("<script>")

    # JavaScript array of friendly labels for the x-axis
    def arr_js_str(string_list: List[str]) -> str:
        return "[" + ",".join(f"'{s}'" for s in string_list) + "]"

    # Main metrics
    main_labels_js = arr_js_str(friendly_phases)
    centers_js = "[" + ",".join(str(x) for x in centers_arr) + "]"
    units_js = "[" + ",".join(str(x) for x in units_arr) + "]"
    infl_js = "[" + ",".join(str(x) for x in influence_arr) + "]"
    builds_js = "[" + ",".join(str(x) for x in builds_arr) + "]"
    civdis_js = "[" + ",".join(str(x) for x in civdis_arr) + "]"

    html.append(f"""
    var ctxMain = document.getElementById('mainMetricsChart').getContext('2d');
    var mainMetricsChart = new Chart(ctxMain, {{
      type: 'line',
      data: {{
        labels: {main_labels_js},
        datasets: [
          {{
            label: 'Centers',
            data: {centers_js},
            borderColor: 'red',
            fill: false
          }},
          {{
            label: 'Units',
            data: {units_js},
            borderColor: 'blue',
            fill: false
          }},
          {{
            label: 'Influence',
            data: {infl_js},
            borderColor: 'green',
            fill: false
          }},
          {{
            label: 'Builds',
            data: {builds_js},
            borderColor: 'purple',
            fill: false
          }},
          {{
            label: 'Civil Disorder',
            data: {civdis_js},
            borderColor: 'orange',
            fill: false
          }}
        ]
      }},
      options: {{
        responsive: true,
        scales: {{
          y: {{
            beginAtZero: true
          }}
        }}
      }}
    }});
""")

    # Relationship charts
    color_palette = [
        "maroon","steelblue","darkgreen","purple","darkorange",
        "navy","fuchsia","darkcyan","brown","teal","black"
    ]
    def build_rel_datasets(
        data_map: Dict[str, List[int]], 
        keys: List[str], 
        extra_line: List[float], 
        extra_label: str
    ) -> str:
        lines = []
        idx = 0
        for k in keys:
            color = color_palette[idx % len(color_palette)]
            vals = data_map[k]
            vals_js = "[" + ",".join(str(v) for v in vals) + "]"
            lines.append(f"""
            {{
              label: '{k}',
              data: {vals_js},
              borderColor: '{color}',
              fill: false
            }}
            """)
            idx += 1
        # aggregator line
        agg_js = "[" + ",".join(f"{v}" for v in extra_line) + "]"
        lines.append(f"""
        {{
          label: '{extra_label}',
          data: {agg_js},
          borderColor: 'black',
          borderDash: [5,5],
          fill: false
        }}
        """)
        return ",".join(lines)

    # Self relationships
    self_rel_ds = build_rel_datasets(self_rels_data, all_self_keys, self_agg_arr, "Aggregate Self")
    html.append(f"""
    var ctxSelf = document.getElementById('selfRelsChart').getContext('2d');
    var selfRelsChart = new Chart(ctxSelf, {{
      type: 'line',
      data: {{
        labels: {main_labels_js},
        datasets: [{self_rel_ds}]
      }},
      options: {{
        responsive: true,
        scales: {{
          y: {{
            min: -2,
            max: 2,
            ticks: {{
              stepSize: 1
            }}
          }}
        }}
      }}
    }});
""")

    # Others relationships
    others_rel_ds = build_rel_datasets(others_rels_data, all_others_keys, others_agg_arr, "Aggregate Others")
    html.append(f"""
    var ctxOthers = document.getElementById('othersRelsChart').getContext('2d');
    var othersRelsChart = new Chart(ctxOthers, {{
      type: 'line',
      data: {{
        labels: {main_labels_js},
        datasets: [{others_rel_ds}]
      }},
      options: {{
        responsive: true,
        scales: {{
          y: {{
            min: -2,
            max: 2,
            ticks: {{
              stepSize: 1
            }}
          }}
        }}
      }}
    }});
""")

    # Orders chart
    issued_js = "[" + ",".join(str(x) for x in orders_issued_arr) + "]"
    valid_js = "[" + ",".join(str(x) for x in orders_valid_arr) + "]"
    rej_js = "[" + ",".join(str(x) for x in orders_rejected_arr) + "]"

    html.append(f"""
    var ctxOrders = document.getElementById('ordersChart').getContext('2d');
    var ordersChart = new Chart(ctxOrders, {{
      type: 'line',
      data: {{
        labels: {main_labels_js},
        datasets: [
          {{
            label: 'Issued Orders',
            data: {issued_js},
            borderColor: 'red',
            fill: false
          }},
          {{
            label: 'Valid Orders',
            data: {valid_js},
            borderColor: 'green',
            fill: false
          }},
          {{
            label: 'Rejected Orders',
            data: {rej_js},
            borderColor: 'orange',
            fill: false
          }}
        ]
      }},
      options: {{
        responsive: true,
        scales: {{
          y: {{
            beginAtZero: true
          }}
        }}
      }}
    }});
""")

    # If we have SVGs, include the JS to animate them
    if svg_files:
        js_code = """
    document.addEventListener('DOMContentLoaded', function() {
        const svgFiles = SVG_FILES_PLACEHOLDER;
        const phaseLabels = PHASE_LABELS_PLACEHOLDER;
        let currentIndex = 0;
        let isPlaying = false;
        let playInterval = null;
        const playbackSpeed = 1500;
        const preloadedSVGs = [];
        
        const svgDisplay = document.getElementById('svgDisplay');
        const prevBtn = document.getElementById('prevBtn');
        const playPauseBtn = document.getElementById('playPauseBtn');
        const nextBtn = document.getElementById('nextBtn');
        const progressBar = document.getElementById('progressBar');
        const progressContainer = document.getElementById('progressContainer');
        const phaseLabel = document.getElementById('phaseLabel');
        
        svgDisplay.style.position = 'relative';
        
        function preloadSVGs() {
            svgFiles.forEach((file, index) => {
                const img = new Image();
                img.src = file;
                preloadedSVGs[index] = img;
            });
        }
        
        function initPlayer() {
            if (svgFiles.length === 0) return;
            preloadSVGs();
            loadSVG(0);
            updateControls();
            
            prevBtn.style.display = 'none';
            nextBtn.style.display = 'none';
            
            playPauseBtn.addEventListener('click', togglePlayback);
            progressContainer.addEventListener('click', handleProgressClick);
        }
        
        function loadSVG(index) {
            if (index < 0 || index >= svgFiles.length) return;
            currentIndex = index;
            const filePath = svgFiles[currentIndex];
            
            const newObj = document.createElement('object');
            newObj.type = 'image/svg+xml';
            newObj.data = filePath;
            newObj.style.position = 'absolute';
            newObj.style.top = 0;
            newObj.style.left = 0;
            newObj.style.width = '100%';
            newObj.style.height = '100%';
            newObj.style.opacity = '0';
            
            svgDisplay.appendChild(newObj);
            
            newObj.onload = function() {
                newObj.style.transition = 'opacity 0.3s';
                newObj.style.opacity = '1';
                phaseLabel.textContent = phaseLabels[currentIndex];
                const progress = (currentIndex / (svgFiles.length - 1)) * 100;
                progressBar.style.width = progress + '%';
                
                updateControls();
            };
            
            newObj.addEventListener('transitionend', function() {
                Array.from(svgDisplay.children)
                    .filter(child => child !== newObj)
                    .forEach(child => svgDisplay.removeChild(child));
            });
        }
        
        function showPrevious() {
            if (currentIndex > 0) {
                loadSVG(currentIndex - 1);
            }
        }
        
        function showNext() {
            if (currentIndex < svgFiles.length - 1) {
                loadSVG(currentIndex + 1);
            }
        }
        
        function togglePlayback() {
            isPlaying = !isPlaying;
            if (isPlaying) {
                playPauseBtn.textContent = '⏸️';
                playInterval = setInterval(() => {
                    if (currentIndex < svgFiles.length - 1) {
                        loadSVG(currentIndex + 1);
                    } else {
                        clearInterval(playInterval);
                        isPlaying = false;
                        playPauseBtn.textContent = '▶️';
                    }
                }, playbackSpeed);
            } else {
                playPauseBtn.textContent = '▶️';
                clearInterval(playInterval);
            }
        }
        
        function handleProgressClick(e) {
            const rect = progressContainer.getBoundingClientRect();
            const clickPosition = e.clientX - rect.left;
            const containerWidth = rect.width;
            const clickRatio = clickPosition / containerWidth;
            const newIndex = Math.round(clickRatio * (svgFiles.length - 1));
            loadSVG(newIndex);
            //if (isPlaying) {
            //    togglePlayback();
            //}
        }
        
        function updateControls() { }
        
        if (svgDisplay) {
            initPlayer();
        }
        
        var animationTabTrigger = document.getElementById('animation-tab');
        if (animationTabTrigger) {
            animationTabTrigger.addEventListener('shown.bs.tab', function(e) {
                if (!isPlaying) {
                    togglePlayback();
                }
            });
            animationTabTrigger.addEventListener('hidden.bs.tab', function(e) {
                if (isPlaying) {
                    togglePlayback();
                }
            });
        }
    });
    """
        svg_paths_js = "[" + ",".join(f"'{x}'" for x in svg_files) + "]"
        # We already built svg_phases_js
        js_code = js_code.replace("SVG_FILES_PLACEHOLDER", svg_paths_js)
        js_code = js_code.replace("PHASE_LABELS_PLACEHOLDER", svg_phases_js)
        html.append(js_code)

    html.append("""
</script>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body></html>
""")

    return "\n".join(html)



###############################################################################
# 4) Main
###############################################################################
if __name__ == "__main__":
    #run_id = 'o1-1.31'
    #run_id = 'o3-mini-1.32'
    #run_id = 'o3-mini-1.34'
    #run_id = 'gemini-pro-1.5-1.35'
    #run_id = 'claude-3.5-sonnet-20240620-1.36'
    #run_id = 'claude-3.5-sonnet-1.39'
    #run_id = 'grok-2-1212-1.42'
    #run_id = 'gemini-2.0-flash-001-1.44'
    #run_id = 'o3-mini-1.48'
    #run_id = 'o1-1.50'
    #run_id = 'gemini-2.0-flash-001-1.51'
    #run_id = 'o3-mini-high-1.53'
    #run_id = 'claude-3.7-sonnet-1.54'
    run_id = 'deepseek-r1-1.56'
    
    infile = run_id + '.json'

    with open(infile, "r", encoding="utf-8") as f:
        game_data = json.load(f)

    # 1) Parse stats
    stats_for_aut = parse_aut_statistics(game_data)

    # 2) Parse negotiations (with the same data we have about phases so we can label them)
    aut_negs = parse_aut_negotiations(game_data, stats_for_aut)

    # 3) Generate HTML
    html_report = generate_html_report(game_data, stats_for_aut, aut_negs)

    # 4) Write out
    outfile = run_id + '.html'
    with open(outfile, "w", encoding="utf-8") as f:
        f.write(html_report)

    print(f"HTML report generated: {outfile}")


HTML report generated: deepseek-r1-1.56.html
